In [ ]:
from torchvision import models
import torch

import pickle as pk # to save list as file
import joblib # Another way to sale/load model and data

import gc # garbage collector, can help you free memory
!pip install import_ipynb
import import_ipynb # tool allow you to import from .ipynb files. Usualy import only from .py files.
from os.path import exists



from sklearn.ensemble import RandomForestClassifier , ExtraTreesClassifier
from sklearn.metrics import hamming_loss
from sklearn.model_selection import train_test_split 



# torch can use CUDA, this line check if CUDA is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
from create_dataset import DicomDataset, DicomTestDataset # my dataset class
from create_dataset import train_metadata_df # import dataset csv

In [ ]:
# Creating instance of DicomDataset
from torchvision.transforms import ToTensor, ToPILImage, Compose
from torchvision.transforms import RandAugment # makes troubles with dataloader
from torchvision.transforms import AutoAugmentPolicy, AutoAugment

from torchvision.transforms import RandomRotation, RandomPerspective, RandomAffine, RandomAdjustSharpness

rand_aug_transformation = Compose([
    ToTensor(),
    #AutoAugment(AutoAugmentPolicy.IMAGENET)
    #RandomRotation(degrees=(-90,90)),
    RandomAffine(degrees=(-180,180),scale = (0.8, 1.2), translate = (0.1, 0.1)  ),
    RandomAdjustSharpness(sharpness_factor=2),
    RandomPerspective(distortion_scale = 0.2, p = 0.3) 
    ])

# policies = [T.AutoAugmentPolicy.CIFAR10, T.AutoAugmentPolicy.IMAGENET, T.AutoAugmentPolicy.SVHN]
# augmenters = [T.AutoAugment(policy) for policy in policies]
# imgs = [
#     [augmenter(orig_img) for _ in range(4)]
#     for augmenter in augmenters
# ]

train_dataset = DicomDataset(
    merged_df=train_metadata_df,
    image_dir="train",
#    train=True,
#    download=True,
    preload = True,
    transform=rand_aug_transformation
)



# test_dataset = DicomTestDataset(
#     image_dir="test",
# #    train=True,
# #    download=True,
#     preload = False,
#     transform=ToTensor()
# )

# test_augmented_dataset = DicomTestDataset(
#     image_dir="test",
# #    train=True,
# #    download=True,
#     preload = False,
#     transform=rand_aug_transformation
# )

# >>> transforms.Compose([
# >>>     transforms.CenterCrop(10),
# >>>     transforms.PILToTensor(),
# >>>     transforms.ConvertImageDtype(torch.float),
# >>> ])
#augmenter = T.RandAugment()


# train_size = int(0.8 * len(training_data))
# test_size = len(training_data) - train_size
# train_part, test_part = torch.utils.data.random_split(training_data, [train_size, test_size])
# train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import random_split


# batch_size=32 ok for alexnet&resnet. Too big for densenet. let's try smaller size
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)
# test_augmented_dataloader = DataLoader(test_augmented_dataset, batch_size=32, shuffle=True)

train_dataloader.__iter__()

In [ ]:
# for list_of_tensor_images,list_of_labels in train_dataloader:
#     print(list_of_tensor_images.shape,"LABELS !!!!",list_of_labels)
#     break

# for list_of_tensor_images,list_of_labels in test_dataloader:
#     print(list_of_tensor_images.shape,"LABELS !!!!",list_of_labels)
#     break

# for list_of_tensor_images,list_of_labels in test_augmented_dataloader:
#     print(list_of_tensor_images.shape,"LABELS !!!!",list_of_labels)
#     break


In [ ]:
torch.cuda.empty_cache()

In [ ]:
#model = models.alexnet(pretrained=True)
# fit should not be used it cycle. This fuction needs more work.
def train_trees_with_nn(tree_name,nn_name, n_epoch = 10,prefix = '', suffix = ''):
    print(f'Starting {nn_name} for {tree_name}\n')
    tree_filename = prefix + tree_name +"_"+ nn_name + suffix + ".tre"
    gc.collect()
    torch.cuda.empty_cache()
    #model = model_dict[model_name]
    model = create_model(nn_name)
    model.eval()
    model.to(device)

    ds_res= []
    tree = create_tree(tree_name,n_estimators= 100000)
    if (exists(tree_filename) ):
        with open(tree_filename, 'rb') as f:
            tree = Pickle.load(f)

    #print("start ",model_name)
    #for image,lable in data_set:
    for epoch in range(n_epoch):
        print(f"Starting epoch ={epoch}\n")
        
        for list_of_tensor_images,list_of_labels in (train_dataloader):
            print("And ",len(list_of_labels)," items more, baby")
            torch.cuda.empty_cache()
            with torch.no_grad():
                features = model(list_of_tensor_images.to(device))
                list_of_tensor_images.cpu()
                del list_of_tensor_images
                #features = model(torch.unsqueeze(image,0).to(device))
                #print(type(features),type(features.cpu()),features.cpu().shape,len([ elem.cpu() for elem in features ]),features.shape)

            
            #alex_np_fea = [ elem.numpy()[0] for elem in alex_features ]
            sk_feat = [ elem.numpy() for elem in features.cpu() ]
            #print(len(sk_feat),len(sk_feat[0]),sk_feat)
            #print(len(list_of_labels),len(list_of_labels[0]),list_of_labels)
            
            tree.fit(sk_feat,list_of_labels)
            #tree_pred = tree.predict(sk_feat)
            #print(hamming_loss(tree_pred,list_of_labels) ) # Boolshit, test on same data as train . But i need to see if tree touches all data
            #ds_res.append(im_res.to('cpu'))
            del features
            del sk_feat
            torch.cuda.empty_cache()
            gc.collect()

        with open(tree_filename, 'wb') as f: # Save tree data every epoch
            Pickle.dump(tree, f)
    del model
    del tree
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
model_names = ["alexnet","resnet50", 
"densenet161", 
"wide_resnet50_2",
"vgg13"]
tree_names = ["RandomForest",
            "ExtraTrees"]

def create_model(model_name):
    model_dict = {"alexnet":models.alexnet(pretrained=True),"resnet50":models.resnet50(pretrained=True), 
    "densenet161":models.densenet161(pretrained=True), 
    "wide_resnet50_2":models.wide_resnet50_2(pretrained=True),
    "vgg13":models.vgg13(pretrained=True)}
    return model_dict[model_name]
    


def create_tree(tree_name, n_estimators = 10000):
    trees_ensembles = { "RandomForest":RandomForestClassifier(n_jobs=8,verbose=1, n_estimators= n_estimators),
            "ExtraTrees":ExtraTreesClassifier(n_jobs=8,verbose=1, n_estimators= n_estimators)}
    return trees_ensembles[tree_name]


In [ ]:


def train_trees_with_nn_in_cache(tree_name,nn_name, n_epoch = 10,prefix = '', suffix = ''):
    print(f'Starting {nn_name} for {tree_name}\n')
    tree_filename = prefix + tree_name +"_"+ nn_name + suffix + ".tre"
    gc.collect()
    torch.cuda.empty_cache()
    #model = model_dict[model_name]
    model = create_model(nn_name)
    model.eval()
    model.to(device)
    #final_in_features = model.fc.in_features
    #model.fc = nn.Linear(final_in_features, 200)



    
    tree = create_tree(tree_name,n_estimators= 20000)
    if (exists(tree_filename) ):
        tree =joblib.load(tree_filename) # joblib way

        # with open(tree_filename, 'rb') as f: # pickle way
        #     tree = Pickle.load(f)
    ds_res= []
    ds_lab = []
    #print("start ",model_name)
    #for image,lable in data_set:
    for epoch in range(n_epoch):
        print(f"Starting epoch ={epoch}\n")
        
        batch_counter = 0
        for list_of_tensor_images,list_of_labels in (train_dataloader):
            print("And ",len(list_of_labels)," items more, baby. Batch number ",batch_counter)
            batch_counter += 1
            torch.cuda.empty_cache()
            with torch.no_grad():
                features = model(list_of_tensor_images.to(device))
                list_of_tensor_images.cpu()
                del list_of_tensor_images
                #features = model(torch.unsqueeze(image,0).to(device))
                #print(type(features),type(features.cpu()),features.cpu().shape,len([ elem.cpu() for elem in features ]),features.shape)

            
            #alex_np_fea = [ elem.numpy()[0] for elem in alex_features ]
            sk_feat = [ elem.numpy() for elem in features.cpu() ]
            sk_labs = [ elem.numpy() for elem in list_of_labels.cpu() ]
            ds_res.extend(sk_feat)
            ds_lab.extend(sk_labs)
            #print(len(sk_feat),len(sk_feat[0]),sk_feat)
            #print(len(list_of_labels),len(list_of_labels[0]),list_of_labels)
            
            #tree.fit(sk_feat,list_of_labels)
            #tree_pred = tree.predict(sk_feat)
            #print(hamming_loss(tree_pred,list_of_labels) ) # Boolshit, test on same data as train . But i need to see if tree touches all data
            #ds_res.append(im_res.to('cpu'))

            #print(ds_res,ds_lab)
            #tree.fit(ds_res,ds_lab)
            del features
            del sk_feat
            del sk_labs
            torch.cuda.empty_cache()
            gc.collect()
        #print(f"Teaching {tree_name} after epoch {epoch} with features from {nn_name}")
        #X_train, X_test, y_train, y_test = train_test_split(ds_res, ds_lab, test_size=0.33, random_state=42)
        #tree.fit(X_train,y_train)
        #tree_pred = tree.predict(X_test)
        #print("CUrrent hamming loss is ",hamming_loss(tree_pred,y_test) )

    print("final education")
    tree.fit(ds_res,ds_lab)
    del ds_res
    del ds_lab
    torch.cuda.empty_cache()
    gc.collect()

    joblib.dump(tree, tree_filename, compress = 3)
    # with open(tree_filename, 'wb') as f: # pickle way
    #     Pickle.dump(tree, f)            
        
    del model
    del tree
    torch.cuda.empty_cache()
    gc.collect()



                

In [ ]:
# for model in model_names:
#     for tree in tree_names:
#         train_trees_with_nn_in_cache(tree,model, n_epoch = 20,prefix = '', suffix = '')

In [ ]:

# Starting densenet161 features preparing -- cannot use batch size = batch_size=32

def save_augmented_nn(nn_name, n_epoch = 10,prefix = '', suffix = ''):
    print(f'Starting {nn_name} features preparing\n')
    #tree_filename = prefix + tree_name +"_"+ nn_name + suffix + ".tre"
    features_filename = prefix + nn_name + suffix + ".fea"
    labels_filename = prefix + nn_name + suffix + ".labels"
    gc.collect()
    torch.cuda.empty_cache()
    #model = model_dict[model_name]
    model = create_model(nn_name)
    model.eval()
    model.to(device)
    
    ds_res= []
    ds_lab = []
    #print("start ",model_name)
    #for image,lable in data_set:
    for epoch in range(n_epoch):
        print(f"Starting epoch ={epoch}\n")
        
        batch_counter = 0
        for list_of_tensor_images,list_of_labels in (train_dataloader):
            print("And ",len(list_of_labels)," items more, baby. Batch number ",batch_counter)
            batch_counter += 1
            torch.cuda.empty_cache()
            with torch.no_grad():
                features = model(list_of_tensor_images.to(device))
                list_of_tensor_images.cpu()
                del list_of_tensor_images
                #features = model(torch.unsqueeze(image,0).to(device))
                #print(type(features),type(features.cpu()),features.cpu().shape,len([ elem.cpu() for elem in features ]),features.shape)

            
            #alex_np_fea = [ elem.numpy()[0] for elem in alex_features ]
            sk_feat = [ elem.numpy() for elem in features.cpu() ]
            sk_labs = [ elem.numpy() for elem in list_of_labels.cpu() ]
            ds_res.extend(sk_feat)
            ds_lab.extend(sk_labs)
            #print(len(sk_feat),len(sk_feat[0]),sk_feat)
            
            del features
            del sk_feat
            del sk_labs
            torch.cuda.empty_cache()
            gc.collect()
        
    print(f"Saving features and labels to files {features_filename} and {labels_filename}")
    joblib.dump(ds_res, features_filename, compress = 3)
    joblib.dump(ds_lab, labels_filename, compress = 3)
    del ds_res
    del ds_lab
    torch.cuda.empty_cache()
    gc.collect()

        
    del model
    torch.cuda.empty_cache()
    gc.collect()



# model_names = ["alexnet","resnet50", 
# "densenet161", 
# "wide_resnet50_2",
# "vgg13"]


# removed alex & resnet. 
model_names = [
"densenet161", 
"wide_resnet50_2",
"vgg13"]

recalc_features = True
if recalc_features:
    for model in model_names:
        save_augmented_nn(model, n_epoch = 20,prefix = '', suffix = '')            

In [ ]:
# import cPickle

# rf = RandomForestRegresor()
# rf.fit(X, y)

# with open('path/to/file', 'wb') as f:
#     cPickle.dump(rf, f)


# # in your prediction file                                                                                                                                                                                                           

# with open('path/to/file', 'rb') as f:
#     rf = cPickle.load(f)


# preds = rf.predict(new_X)
